In [8]:
from ast import arguments
import numpy as np
import queue
from threading import Thread, Lock
import time
from func_timeout import func_timeout

lock = Lock()


def flatten(t):
    """Flatten a list of lists, e.g. [[a,b],[c]] --> [a,b,c] """
    return [
        item for sublist in t if sublist != None for item in sublist
        if item != None
    ]


def _process_multi_func(idx_args_funcs: list, verbose: bool, timeout: int | None, retries: int) -> list | None:
    results: list = []
    iden = np.random.randint(1000)
    # print(f"Thread {iden} started")
    # as long as we have function calls remaining
    while len(idx_args_funcs) > 0:
        with lock:
            idx, argument, func = idx_args_funcs.pop(-1)
        try:
            if timeout is None:
                result = func(*argument) # get result of function call)
            else:
                try:
                    result = func_timeout(timeout, func, argument)
                except:
                    raise Exception(f"The function call for args {argument} has time out.")
            if verbose:
                print(f"Function call {idx} successful with result {result}")
        except: # notify user that call has failed
            result = None 
            print(f"Function call {idx} has failed")
            pass
        
        
        if result is not None:
            results.append((idx, argument, result))

    if len(results) >= 1: # return results if there are any
        return results


def process_parallel_multifunc(funcs: callable, arguments: list[list], timeout: float | None = None, 
                     retries: int = 1, n_threads: int = 5, verbose=False) -> tuple[list[list], list]:
    """ This function created multiple (n_thread) threads and processes them in parallel. It
    takes in a function & a list of arguments and distributes the function calls (with the respective
    arguments) across those threads. Especially useful when dealing with e.g. scraping where you might 
    have a limit on the number of threads that can be run in parallel.

    Args:
        funcs (list[executable]): list of functions used for processing
        arguments (list[list]): list of arguments that should be passed to the function ([[arg1, arg2], [arg1, arg2], ...])
        retries (int, optional): number of retries for each function. Useful for scraping & other functions that may occasionally fail Defaults to 1.
        n_threads (int, optional): number of threads to use. Defaults to 5.
        verbose (bool, optional): verbosity of the function. true = printout, false = silent. Defaults to False.

    Returns:
        tuple[list[list], list]: _description_
    """

    indices = list(
        np.arange(len(arguments))
    )  # indeces are needed to keep track of which order things should be returned in
    idx_args_funcs = [(idx, arg, func) for idx, arg, func in zip(indices, arguments, funcs)]
    threads_list = []  # list to store the different threads
    que = queue.Queue()  # queure from which the threads take their data

    # creates desired number of threads
    for _ in range(n_threads):
        threads_list.append(
            Thread(target=lambda q, arg1, arg2: q.put(
                _process_multi_func(idx_args_funcs, verbose, timeout, retries)),
                   args=(que, indices, arguments)))
        threads_list[-1].start()

    # waits until all data is processed
    time.sleep(1)
    while len(idx_args_funcs) > 1:
        time.sleep(0.001) # on some systems (e.g. Mac M1 Pro chips) if no time.sleep is included the while loop slows down
        # the threaded function calls significantly. No idea why but oh well...
        pass

    for thread in threads_list:
        thread.join()

    if verbose:
        print("Threads joined")
    results = []

    while not que.empty():
        results.append(que.get())
    print("Results complete")

    results = flatten(results)
    sorted_results = sorted(results, key=lambda tup: tup[0])
    print(sorted_results)
    
    return [i[1] for i in sorted_results], [i[2] for i in sorted_results]

from unittest import result
from func_timeout import func_timeout

In [9]:
funcA = lambda x,y: x*y
funcB = lambda x,y: x+y


args = [[1,2], [3,4], [5,6], [7,8]]
funcs = [funcA, funcB, funcA, funcB]



results = process_parallel(funcs, args, timeout = 5)

Results complete
[(0, [1, 2], 2), (1, [3, 4], 7), (2, [5, 6], 30), (3, [7, 8], 15)]


In [10]:
results

([[1, 2], [3, 4], [5, 6], [7, 8]], [2, 7, 30, 15])